# Spotify Tracks Test Notebook

In [ ]:
def get_random_track_id(data):
    return data['track_id'].sample().iloc[0]

In [ ]:
def get_predictions(model, features):
    return model.predict(features).flatten()

In [ ]:
def recommend_system(track_id, data, columns, track_lookup, model, top_n=5):
    if track_id not in data['track_id'].values:
        return "Track ID not found in data."
    
    data.fillna(0, inplace=True)

    features = data[columns].astype('float32').values
    
    track_idx = np.where(data['track_id'] == track_id)[0][0]
    track_features = features[track_idx].reshape(1, -1)

    if track_features.shape[1] != model.layers[0].input_shape[1]:
        return f"Feature mismatch: Model expects {model.layers[0].input_shape[1]} features, but track_features has {track_features.shape[1]}."

    model_predictions = get_predictions(model, features)
    track_prediction = get_predictions(model, track_features)

    similarity_scores = cosine_similarity(track_prediction.reshape(1, -1), model_predictions.reshape(-1, 1))
    similar_indices = np.argsort(similarity_scores[0])[::-1][1:top_n+1]

    recommended_tracks = data.iloc[similar_indices]
    recommended_tracks = recommended_tracks.drop_duplicates()
    recommended_tracks['model_prediction'] = model_predictions[similar_indices]

    recommended_tracks = pd.merge(recommended_tracks, track_lookup, on='track_id', how='left')
    track_info = recommended_tracks[['track_name_y', 'album_name_y', 'artists_y', 'model_prediction']]
    top_track_info = track_info.head(min(5, len(track_info)))

    return top_track_info